In [1]:
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib
import glob
from sklearn import cluster

In [2]:
videoDF = pd.read_csv('D:/dataset/youtube_faces_with_keypoints_small.csv')
videoDF.head(15)

FileNotFoundError: File b'D:/dataset/youtube_faces_with_keypoints_small.csv' does not exist

In [ ]:
# create a dictionary that maps videoIDs to full file paths
npzFilesFullPath = glob.glob('D:/dataset/youtube_faces_*/*.npz')

videoIDs = [x.split("\\")[-1].split('.')[0] for x in npzFilesFullPath]
fullPaths = {}

for videoID, fullPath in zip(videoIDs, npzFilesFullPath):
    fullPaths[videoID] = fullPath

# remove from the large csv file all videos that weren't uploaded yet
videoDF = videoDF.loc[videoDF.loc[:,'videoID'].isin(fullPaths.keys()),:].reset_index(drop=True)
#print(videoDF)
print('Number of Videos is %d' %(videoDF.shape[0]))
print('Number of Unique Individuals is %d' %(len(videoDF['personName'].unique())))

In [ ]:
videoDF = pd.read_csv('D:/dataset/youtube_faces_with_keypoints_large.csv')
videoDF.head(15)

In [ ]:
# create a dictionary that maps videoIDs to full file paths
npzFilesFullPath = glob.glob('D:/dataset/youtube_faces_*/*.npz')

videoIDs = [x.split("\\")[-1].split('.')[0] for x in npzFilesFullPath]
fullPaths = {}

for videoID, fullPath in zip(videoIDs, npzFilesFullPath):
    fullPaths[videoID] = fullPath

# remove from the large csv file all videos that weren't uploaded yet
videoDF = videoDF.loc[videoDF.loc[:,'videoID'].isin(fullPaths.keys()),:].reset_index(drop=True)
#print(videoDF)
print('Number of Videos is %d' %(videoDF.shape[0]))
print('Number of Unique Individuals is %d' %(len(videoDF['personName'].unique())))

In [ ]:
# overview of the contents of the dataset
groupedByPerson = videoDF.groupby("personName")
numVidsPerPerson = groupedByPerson.count()['videoID']
groupedByPerson.count().sort_values('videoID', axis=0, ascending=False)

plt.close('all')
plt.figure(figsize=(25,20))
plt.subplot(2,2,1)
plt.hist(x=numVidsPerPerson,bins=0.5+np.arange(numVidsPerPerson.min()-1,numVidsPerPerson.max()+1))
plt.title('Number of Videos per Person',fontsize=30); 
plt.xlabel('Number of Videos',fontsize=25); plt.ylabel('Number of People',fontsize=25)

plt.subplot(2,2,2)
plt.hist(x=videoDF['videoDuration'],bins=28);
plt.title('Distribution of Video Duration',fontsize=30); 
plt.xlabel('duration [frames]',fontsize=25); plt.ylabel('Number of Videos',fontsize=25)
plt.xlim(videoDF['videoDuration'].min()-2,videoDF['videoDuration'].max()+2)

plt.subplot(2,2,3)
plt.scatter(x=videoDF['imageWidth'], y=videoDF['imageHeight'])
plt.title('Distribution of Image Sizes',fontsize=30)
plt.xlabel('Image Width [pixels]',fontsize=25); plt.ylabel('Image Height [pixels]',fontsize=25)
plt.xlim(0,videoDF['imageWidth'].max() +15)
plt.ylim(0,videoDF['imageHeight'].max()+15)

plt.subplot(2,2,4)
averageFaceSize_withoutNaNs = np.array(videoDF['averageFaceSize'])
averageFaceSize_withoutNaNs = averageFaceSize_withoutNaNs[np.logical_not(np.isnan(averageFaceSize_withoutNaNs))]
plt.hist(averageFaceSize_withoutNaNs, bins=28)
plt.title('Distribution of Average Face Sizes ',fontsize=30)
plt.xlabel('Average Face Size [pixels]',fontsize=25); plt.ylabel('Number of Videos',fontsize=25);


In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('8FdSHl4oNIM',width=640, height=480)

In [ ]:
# show several frames from each video and overlay 2D keypoints
np.random.seed(3)
numVideos = 4
framesToShowFromVideo = np.array([0.1,0.5,0.9])
numFramesPerVideo = len(framesToShowFromVideo)

# define which points need to be connected with a line
jawPoints          = [ 0,17]
rigthEyebrowPoints = [17,22]
leftEyebrowPoints  = [22,27]
noseRidgePoints    = [27,31]
noseBasePoints     = [31,36]
rightEyePoints     = [36,42]
leftEyePoints      = [42,48]
outerMouthPoints   = [48,60]
innerMouthPoints   = [60,68]

listOfAllConnectedPoints = [jawPoints,rigthEyebrowPoints,leftEyebrowPoints,
                            noseRidgePoints,noseBasePoints,
                            rightEyePoints,leftEyePoints,outerMouthPoints,innerMouthPoints]

# select a random subset of 'numVideos' from the available videos
randVideoIDs = videoDF.loc[np.random.choice(videoDF.index,size=numVideos,replace=False),'videoID']

fig, axArray = plt.subplots(nrows=numVideos,ncols=numFramesPerVideo,figsize=(14,18))
for i, videoID in enumerate(randVideoIDs):
    # load video
    videoFile = np.load(fullPaths[videoID])
    colorImages = videoFile['colorImages']
    boundingBox = videoFile['boundingBox']
    landmarks2D = videoFile['landmarks2D']
    landmarks3D = videoFile['landmarks3D']

    # select frames and show their content
    selectedFrames = (framesToShowFromVideo*(colorImages.shape[3]-1)).astype(int)
    for j, frameInd in enumerate(selectedFrames):
        axArray[i][j].imshow(colorImages[:,:,:,frameInd])
        axArray[i][j].scatter(x=landmarks2D[:,0,frameInd],y=landmarks2D[:,1,frameInd],s=9,c='r')
        for conPts in listOfAllConnectedPoints:
            xPts = landmarks2D[conPts[0]:conPts[-1],0,frameInd]
            yPts = landmarks2D[conPts[0]:conPts[-1],1,frameInd]
        
            if conPts == [36,42] or conPts == [42,48] or conPts == [48,60] or conPts == [60,68]:
                xPts=np.concatenate((xPts, [xPts[0]]))
                yPts=np.concatenate((yPts, [yPts[0]]))
            
            axArray[i][j].plot(xPts,yPts,c='w',lw=1)
        axArray[i][j].set_title('"%s" (t=%d)' %(videoID,frameInd), fontsize=12)
        axArray[i][j].set_axis_off()

In [ ]:
import cv2

In [ ]:
def points_vis():
    # show several frames from each video and overlay 2D keypoints
    #np.random.seed(2)
    numVideos = 5
    framesToShowFromVideo = np.array([0.1,0.5,0.9])
    numFramesPerVideo = len(framesToShowFromVideo)

    # define which points need to be connected with a line
    jawPoints          = [ 0,17]
    rigthEyebrowPoints = [17,22]
    leftEyebrowPoints  = [22,27]
    noseRidgePoints    = [27,31]
    noseBasePoints     = [31,36]
    rightEyePoints     = [36,42]
    leftEyePoints      = [42,48]
    outerMouthPoints   = [48,60]
    innerMouthPoints   = [60,68]

    listOfAllConnectedPoints = [jawPoints,rigthEyebrowPoints,leftEyebrowPoints,
                                noseRidgePoints,noseBasePoints,
                                rightEyePoints,leftEyePoints,outerMouthPoints,innerMouthPoints]

    # select a random subset of 'numVideos' from the available videos
    randVideoIDs = videoDF.loc[np.random.choice(videoDF.index,size=numVideos,replace=False),'videoID']

    fig, axArray = plt.subplots(nrows=numVideos,ncols=numFramesPerVideo,figsize=(14,18))
    for i, videoID in enumerate(randVideoIDs):
        # load video
        videoFile = np.load(fullPaths[videoID])
        colorImages = videoFile['colorImages']
        boundingBox = videoFile['boundingBox']
        landmarks2D = videoFile['landmarks2D']
        landmarks3D = videoFile['landmarks3D']
############################################################

        """selectedFrames = (framesToShowFromVideo*(colorImages.shape[3]-1)).astype(int)
        for j, frameInd in enumerate(selectedFrames):
            
            axArray[i][j].imshow(colorImages[:,:,:,frameInd])
            axArray[i][j].scatter(x=landmarks2D[:,0,frameInd],y=landmarks2D[:,1,frameInd],s=9,c='r')
            for conPts in listOfAllConnectedPoints:
                xPts = landmarks2D[conPts[0]:conPts[-1],0,frameInd]
                yPts = landmarks2D[conPts[0]:conPts[-1],1,frameInd]

                if conPts == [36,42] or conPts == [42,48] or conPts == [48,60] or conPts == [60,68]:
                    xPts=np.concatenate((xPts, [xPts[0]]))
                    yPts=np.concatenate((yPts, [yPts[0]]))

                axArray[i][j].plot(xPts,yPts,c='w',lw=1)
            axArray[i][j].set_title('"%s" (t=%d)' %(videoID,frameInd), fontsize=12)
            axArray[i][j].set_axis_off()
        """
        #######################################


    capture = cv2.VideoCapture(videoFile )

    while (capture.isOpened()):

        ret, frame = capture.read()
        if ret:

            cv2.imshow('Video Stream', frame)



        else:
            break

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    capture.release()
    cv2.destroyAllWindows()


            
#points_vis()

In [ ]:
# show several 3D keypoints
numVideos = 4
framesToShowFromVideo = np.array([0.2,0.5,0.8])
numFramesPerVideo = len(framesToShowFromVideo)

# select a random subset of 'numVideos' from the available videos
randVideoIDs = videoDF.loc[np.random.choice(videoDF.index,size=numVideos,replace=False),'videoID']

fig = plt.figure(figsize=(14,14))
for i, videoID in enumerate(randVideoIDs):
    # load video
    videoFile = np.load(fullPaths[videoID])
    colorImages = videoFile['colorImages']
    boundingBox = videoFile['boundingBox']
    landmarks2D = videoFile['landmarks2D']
    landmarks3D = videoFile['landmarks3D']

    # select frames and show their content
    selectedFrames = (framesToShowFromVideo*(colorImages.shape[3]-1)).astype(int)
    for j, frameInd in enumerate(selectedFrames):
        subplotInd = i*numFramesPerVideo + j+1
        ax = fig.add_subplot(numVideos, numFramesPerVideo, subplotInd, projection='3d')
        ax.scatter(landmarks3D[:,0,frameInd], landmarks3D[:,1,frameInd], landmarks3D[:,2,frameInd],c='r')
        
        ###ax.imshow(inImages[:,:,:,frameInd])# remove this SIWAR if not working
        
        for conPts in listOfAllConnectedPoints:
            xPts = landmarks3D[conPts[0]:conPts[-1],0,frameInd]
            yPts = landmarks3D[conPts[0]:conPts[-1],1,frameInd]
            zPts = landmarks3D[conPts[0]:conPts[-1],2,frameInd]
            if conPts == [36,42] or conPts == [42,48] or conPts == [48,60] or conPts == [60,68]:
                xPts=np.concatenate((xPts, [xPts[0]]))
                yPts=np.concatenate((yPts, [yPts[0]]))
                zPts=np.concatenate((zPts, [zPts[0]]))
                
            
            ax.plot3D(xPts,yPts,zPts,color='g')         
        ax.set_xlim(ax.get_xlim()[::-1])
        ax.view_init(elev=96, azim=90)
        ax.set_title('"%s" (t=%d)' %(videoID,frameInd), fontsize=12)
        
plt.tight_layout()

In [ ]:
# collect all 2D and 3D shapes from all frames from all videos to a single numpy array matrix
totalNumberOfFrames = videoDF['videoDuration'].sum()
landmarks2D_all = np.zeros((68,2,int(totalNumberOfFrames)))
landmarks3D_all = np.zeros((68,3,int(totalNumberOfFrames)))

shapeIndToVideoID = {} # dictionary for later useage
endInd = 0
for i, videoID in enumerate(videoDF['videoID']):
    
    # load video
    videoFile = np.load(fullPaths[videoID])
    landmarks2D = videoFile['landmarks2D']
    landmarks3D = videoFile['landmarks3D']

    startInd = endInd
    endInd   = startInd + landmarks2D.shape[2]

    # store in one big array
    landmarks2D_all[:,:,startInd:endInd] = landmarks2D
    landmarks3D_all[:,:,startInd:endInd] = landmarks3D
    
    # make sure we keep track of the mapping to the original video and frame
    for videoFrameInd, shapeInd in enumerate(range(startInd,endInd)):
        shapeIndToVideoID[shapeInd] = (videoID, videoFrameInd)

# center the shapes around zero
# i.e. such that for each frame the mean x,y,z coordinates will be zero
# or in math terms: Xc = X - mean(X), Yc = Y - mean(Y), Zc = Z - mean(Z)
landmarks2D_centered = np.zeros(landmarks2D_all.shape)
landmarks2D_centered = landmarks2D_all - np.tile(landmarks2D_all.mean(axis=0),[68,1,1])

landmarks3D_centered = np.zeros(landmarks3D_all.shape)
landmarks3D_centered = landmarks3D_all - np.tile(landmarks3D_all.mean(axis=0),[68,1,1])

# normalize the shapes such that they have the same scale
# i.e. such that for each frame the mean euclidian distance from the shape center will be one
# or in math terms: mean( sqrt(dX^2 + dY^2 + dZ^2) ) = 1 
landmarks2D_normlized = np.zeros(landmarks2D_all.shape)
landmarks2D_normlized = landmarks2D_centered / np.tile(np.sqrt((landmarks2D_centered**2).sum(axis=1)).mean(axis=0), [68,2,1])

landmarks3D_normlized = np.zeros(landmarks3D_all.shape)
landmarks3D_normlized = landmarks3D_centered / np.tile(np.sqrt((landmarks3D_centered**2).sum(axis=1)).mean(axis=0), [68,3,1])

In [ ]:
#%% check the 2D normalization and verify that everything is as expected
# select random several frames to be used as test cases
np.random.seed(2)

listOfShapeColors = ['r','g','b','m','y','c','k']
numShapesToPresent = len(listOfShapeColors)
listOfShapeInds = np.random.choice(range(int(totalNumberOfFrames)),size=numShapesToPresent,replace=False)

plt.close('all')
plt.figure(figsize=(14,10))
plt.suptitle('Shape Normalization Stages',fontsize=35)
plt.subplot(1,3,1)
for k,shapeInd in enumerate(listOfShapeInds):
    plt.scatter(landmarks2D_all[:,0,shapeInd], -landmarks2D_all[:,1,shapeInd], s=15, c=listOfShapeColors[k])
    for conPts in listOfAllConnectedPoints:
        xPts =  landmarks2D_all[conPts[0]:conPts[-1],0,shapeInd]
        yPts = -landmarks2D_all[conPts[0]:conPts[-1],1,shapeInd]
        if conPts == [36,42] or conPts == [42,48] or conPts == [48,60] or conPts == [60,68]:
            xPts=np.concatenate((xPts, [xPts[0]]))
            yPts=np.concatenate((yPts, [yPts[0]]))
             
        plt.plot(xPts,yPts,c=listOfShapeColors[k],lw=1)
plt.axis('off'); plt.title('Original Shapes', fontsize=20)

plt.subplot(1,3,2)
for k,shapeInd in enumerate(listOfShapeInds):
    plt.scatter(landmarks2D_centered[:,0,shapeInd], -landmarks2D_centered[:,1,shapeInd], s=15, c=listOfShapeColors[k])
    for conPts in listOfAllConnectedPoints:
        xPts =  landmarks2D_centered[conPts[0]:conPts[-1],0,shapeInd]
        yPts = -landmarks2D_centered[conPts[0]:conPts[-1],1,shapeInd]
        if conPts == [36,42] or conPts == [42,48] or conPts == [48,60] or conPts == [60,68]:
            xPts=np.concatenate((xPts, [xPts[0]]))
            yPts=np.concatenate((yPts, [yPts[0]]))
           
        plt.plot(xPts,yPts,c=listOfShapeColors[k],lw=1)
plt.axis('off'); plt.title('Centered Shapes', fontsize=20)

plt.subplot(1,3,3)
for k,shapeInd in enumerate(listOfShapeInds):
    plt.scatter(landmarks2D_normlized[:,0,shapeInd], -landmarks2D_normlized[:,1,shapeInd], s=15, c=listOfShapeColors[k])
    for conPts in listOfAllConnectedPoints:
        xPts =  landmarks2D_normlized[conPts[0]:conPts[-1],0,shapeInd]
        yPts = -landmarks2D_normlized[conPts[0]:conPts[-1],1,shapeInd]
        if conPts == [36,42] or conPts == [42,48] or conPts == [48,60] or conPts == [60,68]:
            xPts=np.concatenate((xPts, [xPts[0]]))
            yPts=np.concatenate((yPts, [yPts[0]]))
           
        plt.plot(xPts,yPts,c=listOfShapeColors[k],lw=1)
plt.axis('off'); plt.title('Normlized Shapes', fontsize=20)

In [ ]:
#%% cluster normalized shapes and show the cluster centers
numClusters = 16
normalizedShapesTable = np.reshape(landmarks2D_normlized, [68*2, landmarks2D_normlized.shape[2]]).T

shapesModel = cluster.KMeans(n_clusters=numClusters, n_init=5, random_state=1).fit(normalizedShapesTable[::2,:])
clusterAssignment = shapesModel.predict(normalizedShapesTable)

plt.figure(figsize=(14,14))
numRowsAndCols = int(np.ceil(np.sqrt(numClusters)))
for i in range(numClusters):
    plt.subplot(numRowsAndCols,numRowsAndCols,i+1);
    currClusterShape = np.reshape(shapesModel.cluster_centers_[i,:], [68,2])
    plt.scatter(x=currClusterShape[:,0],y=-currClusterShape[:,1],s=20,c='r')
    for conPts in listOfAllConnectedPoints:
        xPts =  currClusterShape[conPts[0]:conPts[-1],0]
        yPts = -currClusterShape[conPts[0]:conPts[-1],1]
        if conPts == [36,42] or conPts == [42,48] or conPts == [48,60] or conPts == [60,68]:
            xPts=np.concatenate((xPts, [xPts[0]]))
            yPts=np.concatenate((yPts, [yPts[0]]))
           
        plt.plot(xPts,yPts,c='g',lw=1)
    plt.title('cluster %d' %(i),fontsize=15)
    plt.axis('off')

In [ ]:
#%% show several original images that are assigned to a particular cluster
selectedCluster = 15
numRows = 4; numCols = 4;

shapeIndsAssignedToCluster = np.nonzero(clusterAssignment == selectedCluster)[0]
listOfShapeInds = np.random.choice(shapeIndsAssignedToCluster ,size=numRows*numCols,replace=False)

plt.figure(figsize=(14,14))
for i, shapeInd in enumerate(listOfShapeInds):
    # load video and pickout the relevent frame
    videoID  = shapeIndToVideoID[shapeInd][0]
    frameInd = shapeIndToVideoID[shapeInd][1]    
    videoFile = np.load(fullPaths[videoID])
    image = videoFile['colorImages'][:,:,:,frameInd]
    
    # show the image
    plt.subplot(numRows,numCols,i+1);
    plt.imshow(image); plt.axis('off')

In [ ]:
#%% show several original images that are assigned to a particular cluster
selectedCluster = 2
numRows = 4; numCols = 4;

shapeIndsAssignedToCluster = np.nonzero(clusterAssignment == selectedCluster)[0]
listOfShapeInds = np.random.choice(shapeIndsAssignedToCluster ,size=numRows*numCols,replace=False)

plt.figure(figsize=(14,14))
for i, shapeInd in enumerate(listOfShapeInds):
    # load video and pickout the relevent frame
    videoID  = shapeIndToVideoID[shapeInd][0]
    frameInd = shapeIndToVideoID[shapeInd][1]    
    videoFile = np.load(fullPaths[videoID])
    image = videoFile['colorImages'][:,:,:,frameInd]
    
    # show the image
    plt.subplot(numRows,numCols,i+1);
    plt.imshow(image); plt.axis('off')

In [ ]:
#%% define shape normalization utility functions
def NormlizeShapes(shapesImCoords):
    (numPoints, numDims, _) = shapesImCoords.shape
    """shapesNomalized, scaleFactors, meanCoords  = NormlizeShapes(shapesImCoords)"""
    
    # calc mean coords and subtract from shapes    
    meanCoords = shapesImCoords.mean(axis=0)
    shapesCentered = np.zeros(shapesImCoords.shape)
    shapesCentered = shapesImCoords - np.tile(meanCoords,[numPoints,1,1])

    # calc scale factors and divide shapes
    scaleFactors = np.sqrt((shapesCentered**2).sum(axis=1)).mean(axis=0)
    shapesNormlized = np.zeros(shapesCentered.shape)
    shapesNormlized = shapesCentered / np.tile(scaleFactors, [numPoints,numDims,1])

    return shapesNormlized, scaleFactors, meanCoords


def TransformShapeBackToImageCoords(shapesNomalized, scaleFactors, meanCoords):
    """shapesImCoords_rec = TransformShapeBackToImageCoords(shapesNomalized, scaleFactors, meanCoords)"""
    (numPoints, numDims, _) = shapesNomalized.shape
    
    # move back to the correct scale
    shapesCentered = shapesNomalized * np.tile(scaleFactors, [numPoints,numDims,1])
    # move back to the correct location
    shapesImCoords = shapesCentered + np.tile(meanCoords,[numPoints,1,1])
    
    return shapesImCoords

In [ ]:
#%% Normalize 2D and 3D shapes

# collect all 2D and 3D shapes from all frames from all videos to a single numpy array matrix
totalNumberOfFrames = videoDF['videoDuration'].sum()
landmarks2D_all = np.zeros((68,2,int(totalNumberOfFrames)))
landmarks3D_all = np.zeros((68,3,int(totalNumberOfFrames)))

shapeIndToVideoID = {} # dictionary for later useage
endInd = 0
for i, videoID in enumerate(videoDF['videoID']):
    
    # load video
    videoFile = np.load(fullPaths[videoID])
    landmarks2D = videoFile['landmarks2D']
    landmarks3D = videoFile['landmarks3D']

    startInd = endInd
    endInd   = startInd + landmarks2D.shape[2]

    # store in one big array
    landmarks2D_all[:,:,startInd:endInd] = landmarks2D
    landmarks3D_all[:,:,startInd:endInd] = landmarks3D
    
    # make sure we keep track of the mapping to the original video and frame
    for videoFrameInd, shapeInd in enumerate(range(startInd,endInd)):
        shapeIndToVideoID[shapeInd] = (videoID, videoFrameInd)

# normlize shapes
landmarks2D_normlized, _, _  = NormlizeShapes(landmarks2D_all)
landmarks3D_normlized, _, _  = NormlizeShapes(landmarks3D_all)

In [ ]:
jawPoints          = np.arange(0,17)
rigthEyebrowPoints = np.arange(17,22)
leftEyebrowPoints  = np.arange(22,27)
noseRidgePoints    = np.arange(27,31)
noseBasePoints     = np.arange(31,36)
rightEyePoints     = np.arange(36,42)
rightEyePoints = np.concatenate((rightEyePoints,[rightEyePoints[0]]))
leftEyePoints      = np.arange(42,48)
leftEyePoints = np.concatenate((leftEyePoints,[leftEyePoints[0]]))
outerMouthPoints   = np.arange(48,60)
outerMouthPoints = np.concatenate((outerMouthPoints,[outerMouthPoints[0]]))
innerMouthPoints   = np.arange(60,68)
innerMouthPoints = np.concatenate((innerMouthPoints,[innerMouthPoints[0]]))


In [ ]:
#%% define a utility function to show 3D animation
def ShowAnimation_3D(landmarks3D):
    
    landmarks3D = landmarks3D.copy()
    landmarks3D[:,0,:] = -landmarks3D[:,0,:]
    
    xMin = landmarks3D[:,0,:].min()-5
    xMax = landmarks3D[:,0,:].max()+5
    yMin = landmarks3D[:,1,:].min()-5
    yMax = landmarks3D[:,1,:].max()+5
    zMin = landmarks3D[:,2,:].min()-5
    zMax = landmarks3D[:,2,:].max()+5
    
    boxCorners = np.array([[xMin,yMin,zMin],
                           [xMin,yMin,zMax],
                           [xMin,yMax,zMin],
                           [xMin,yMax,zMax],
                           [xMax,yMin,zMin],
                           [xMax,yMin,zMax],
                           [xMax,yMax,zMin],
                           [xMax,yMax,zMax]])
    
    traversalOrder = [0,1,3,2,0,4,6,2,6,7,3,7,5,1,5,4]
    boxTraceCoords = np.zeros((len(traversalOrder),3))
    for i, corner in enumerate(traversalOrder):
        boxTraceCoords[i,:] = boxCorners[corner,:]
    
    trace1   = go.Scatter3d(name='Jawline', x=landmarks3D[:,0,1][jawPoints],y=landmarks3D[:,1,1][jawPoints],z=landmarks3D[:,2,1][jawPoints],
                            mode='lines+markers',marker=dict(color = 'blue',opacity=0.7,size = 5))
    
    trace2   = go.Scatter3d(name='Right Eyebrow',x=landmarks3D[:,0,1][rigthEyebrowPoints],y=landmarks3D[:,1,1][rigthEyebrowPoints],z=landmarks3D[:,2,1][rigthEyebrowPoints],
                            mode='lines+markers',marker=dict(color = 'blue',opacity=0.7,size = 5))
    
    trace3   = go.Scatter3d(name='Left Eyebrow',x=landmarks3D[:,0,1][leftEyebrowPoints],y=landmarks3D[:,1,1][leftEyebrowPoints],z=landmarks3D[:,2,1][leftEyebrowPoints],
                            mode='lines+markers',marker=dict(color = 'blue',opacity=0.7,size = 5))
    
    trace4   = go.Scatter3d(name='Nose Ridge',x=landmarks3D[:,0,1][noseRidgePoints],y=landmarks3D[:,1,1][noseRidgePoints],z=landmarks3D[:,2,1][noseRidgePoints],
                            mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
    
    trace5   = go.Scatter3d(name='Nose Base',x=landmarks3D[:,0,1][noseBasePoints],y=landmarks3D[:,1,1][noseBasePoints],z=landmarks3D[:,2,1][noseBasePoints],
                            mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
    
    trace6   = go.Scatter3d(name='Right Eye', x=landmarks3D[:,0,1][rightEyePoints],y=landmarks3D[:,1,1][rightEyePoints],z=landmarks3D[:,2,1][rightEyePoints],
                            mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
    
    trace7   = go.Scatter3d(name='Left Eye', x=landmarks3D[:,0,1][leftEyePoints],y=landmarks3D[:,1,1][leftEyePoints],z=landmarks3D[:,2,1][leftEyePoints],
                            mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
    
    trace8   = go.Scatter3d(name='Outer Mouth', x=landmarks3D[:,0,1][outerMouthPoints],y=landmarks3D[:,1,1][outerMouthPoints],z=landmarks3D[:,2,1][outerMouthPoints],
                            mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
    
    trace9   = go.Scatter3d(name='Inner Mouth', x=landmarks3D[:,0,1][innerMouthPoints],y=landmarks3D[:,1,1][innerMouthPoints],z=landmarks3D[:,2,1][innerMouthPoints],
                            mode='lines+markers',marker=dict(color = 'green',opacity=0.6,size = 5))
        
    boxTrace = go.Scatter3d(name='boundingBox', x=boxTraceCoords[:,0],y=boxTraceCoords[:,1],z=boxTraceCoords[:,2],
                            mode='lines+markers',marker=dict(color = 'red', opacity=1.0,size = 5))
    
    data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7, trace8, trace9, boxTrace]
    
    mfr = []
    for t in range(len(landmarks3D[1,1,:])):
        mfr.append({'data' :[{'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][jawPoints],'y':landmarks3D[:,1,t][jawPoints],'z':landmarks3D[:,2,t][jawPoints]},
                             {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][rigthEyebrowPoints],'y':landmarks3D[:,1,t][rigthEyebrowPoints],'z':landmarks3D[:,2,t][rigthEyebrowPoints]},
                             {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][leftEyebrowPoints],'y':landmarks3D[:,1,t][leftEyebrowPoints],'z':landmarks3D[:,2,t][leftEyebrowPoints]},
                             {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][noseRidgePoints],'y':landmarks3D[:,1,t][noseRidgePoints],'z':landmarks3D[:,2,t][noseRidgePoints]},
                             {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][noseBasePoints],'y':landmarks3D[:,1,t][noseBasePoints],'z':landmarks3D[:,2,t][noseBasePoints]},
                             {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][rightEyePoints],'y':landmarks3D[:,1,t][rightEyePoints],'z':landmarks3D[:,2,t][rightEyePoints]},
                             {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][leftEyePoints],'y':landmarks3D[:,1,t][leftEyePoints],'z':landmarks3D[:,2,t][leftEyePoints]},
                             {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][outerMouthPoints],'y':landmarks3D[:,1,t][outerMouthPoints],'z':landmarks3D[:,2,t][outerMouthPoints]},
                             {'type' : "scatter3d",'mode':'lines+markers',
                              'x':landmarks3D[:,0,t][innerMouthPoints],'y':landmarks3D[:,1,t][innerMouthPoints],'z':landmarks3D[:,2,t][innerMouthPoints]},
                             {'type' : "scatter3d",'mode':'lines+markers',
                             'x':boxTraceCoords[:,0],'y':boxTraceCoords[:,1],'z':boxTraceCoords[:,2]}]})
                                 
    
    layout = go.Layout(width=800, height=800, title='3D Face Shape Animation',
                       scene=dict(camera=dict(up     = dict(x= 0, y=-1.0, z=0),
                                              center = dict(x= 0, y= 0.0, z=0),
                                              eye    = dict(x= 0, y= 0.7, z=2),
                                             )
                                 ),
                        updatemenus=[dict(type='buttons', showactive=False,
                                            y=1,
                                            x=1,
                                            xanchor='right',
                                            yanchor='top',
                                            pad=dict(t=0, r=10),
                                            buttons=[dict(
                                                        label='Play Animation',
                                                        method='animate',
                                                        args=[None, dict(frame       = dict(duration=0.04, redraw=True), 
                                                                         transition  = dict(duration=0),
                                                                         fromcurrent = True,
                                                                         mode = 'immediate'
                                                                        )
                                                             ]
                                                         )
                                                    ]
                                           )
                                      ]
                      )
                                                    
    fig = dict(data=data, layout=layout, frames=mfr)
    py.iplot(fig)

In [ ]:
import numpy as np
import pandas as pd
from sklearn import decomposition
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib
import plotly.offline as py
import plotly.graph_objs as go
#import imageio
import glob

py.init_notebook_mode(connected=True)

In [ ]:
#%% load a 3D landmarks sequence and present it

personToUse ='Ali_Abbas_2'
videoFile = np.load(fullPaths[personToUse])
landmarks3D_curr = videoFile['landmarks3D']

ShowAnimation_3D(landmarks3D_curr)

In [ ]:
#%% build 3D shape model
numComponents = 30

normalizedShapesTable = np.reshape(landmarks3D_normlized, [68*3, landmarks3D_normlized.shape[2]]).T
shapesModel = decomposition.PCA(n_components=numComponents, whiten=True, random_state=1).fit(normalizedShapesTable)
print('Total explained percent by PCA model with %d components is %.1f%s' %(numComponents, 100*shapesModel.explained_variance_ratio_.sum(),'%'))

In [ ]:
#%% interpret the shapes using our shape model (project and reconstruct)

# normlize shapes (and keep the scale factors and mean coords for later reconstruction)
landmarks3D_norm, scaleFactors, meanCoords  = NormlizeShapes(landmarks3D_curr)
# convert to matrix form
landmarks3D_norm_table = np.reshape(landmarks3D_norm, [68*3, landmarks3D_norm.shape[2]]).T
# project onto shapes model and reconstruct
landmarks3D_norm_table_rec = shapesModel.inverse_transform(shapesModel.transform(landmarks3D_norm_table))
# convert back to shapes (numKeypoint, numDims, numFrames)
landmarks3D_norm_rec = np.reshape(landmarks3D_norm_table_rec.T, [68, 3, landmarks3D_norm.shape[2]])
# transform back to image coords
landmarks3D_curr_rec = TransformShapeBackToImageCoords(landmarks3D_norm_rec, scaleFactors, meanCoords)

In [ ]:
# show the new animation
ShowAnimation_3D(landmarks3D_curr_rec)

In [ ]:
#%% plot x(t), y(t), z(t) for several keypoints before and after filtering
selectedKeypointInds    = [ 30, 33,  36, 39,  42, 45,  51, 57,  62, 66,  48, 54]
selectedKeypointColors  = ['g','g', 'r','m', 'm','r', 'b','b', 'g','g', 'y','y']
selectedKeypointStrings = ['nose tip', 'nose base',  'right eye outer','right eye inner',  'left eye inner','left eye outer',
                           'outer mouth top','outer mouth bottom',  'inner mouth top','inner mouth bottom',
                           'right mouth corner','left mouth corner']

plt.figure(figsize=(13,11)); plt.suptitle('Original Traces', fontsize=22)
for subplotInd, yLabel in enumerate(['x(t)','y(t)','z(t)']):
    plt.subplot(3,1,subplotInd+1); plt.ylabel(yLabel,fontsize=20)
    for k,c,legendLabel in zip(selectedKeypointInds,selectedKeypointColors,selectedKeypointStrings):
        plt.plot(landmarks3D_curr[k,subplotInd,:],c=c,label=legendLabel)
    if subplotInd == 0: 
        plt.legend(bbox_to_anchor=(0,1,1,0), shadow=True,
                   loc=3, ncol=4, mode="expand", borderaxespad=0, fontsize=12)
plt.xlabel('time [frame]',fontsize=20);

In [ ]:
plt.figure(figsize=(13,11)); plt.suptitle('Original Vs. "Spatially" Filtered Traces', fontsize=22)
for subplotInd, yLabel in enumerate(['x(t)','y(t)','z(t)']):
    plt.subplot(3,1,subplotInd+1); plt.ylabel(yLabel,fontsize=25)
    plt.plot(landmarks3D_curr[selectedKeypointInds,subplotInd,:].T,c='r')
    plt.plot(landmarks3D_curr_rec[selectedKeypointInds,subplotInd,:].T,c='b')
plt.xlabel('time [frame]',fontsize=20);

In [ ]:
#%% apply temporal filtering on the 3D points and show filtered signals
filterHalfLength = 2
temporalFilter = np.ones((1,1,2*filterHalfLength+1))
temporalFilter = temporalFilter / temporalFilter.sum()

startTileBlock = np.tile(landmarks3D_curr_rec[:,:,0][:,:,np.newaxis],[1,1,filterHalfLength])
endTileBlock = np.tile(landmarks3D_curr_rec[:,:,-1][:,:,np.newaxis],[1,1,filterHalfLength])
landmarks3D_curr_rec_padded = np.dstack((startTileBlock,landmarks3D_curr_rec,endTileBlock))
landmarks3D_curr_rec_filtered = signal.convolve(landmarks3D_curr_rec_padded, temporalFilter, mode='valid', method='fft')

plt.figure(figsize=(13,11)); plt.suptitle('Original Vs. Spatio-Temporally Filtered Traces', fontsize=22)
for subplotInd, yLabel in enumerate(['x(t)','y(t)','z(t)']):
    plt.subplot(3,1,subplotInd+1); plt.ylabel(yLabel,fontsize=20)
    plt.plot(landmarks3D_curr[selectedKeypointInds,subplotInd,:].T,c='r')
    plt.plot(landmarks3D_curr_rec_filtered[selectedKeypointInds,subplotInd,:].T,c='b')
plt.xlabel('time [frame]',fontsize=20);

In [ ]:
#%% show animation of the temporally filtered 3D points
ShowAnimation_3D(landmarks3D_curr_rec_filtered)

In [ ]:
#%% helper function: create videos with keypoints overlaid for each of the 3 processing stages
def CreateVideosWithMarkingsSideBySide(colorImages, landmarks3D_curr, landmarks3D_curr_rec, landmarks3D_curr_rec_filtered):
    imageWithMarkings_orig   = colorImages.copy()
    imageWithMarkings_sp     = colorImages.copy()
    imageWithMarkings_sp_tmp = colorImages.copy()
    
    # paint requested channel
    channelToMark = 1
    for frame in range(colorImages.shape[3]):
        for k in range(landmarks3D_curr.shape[0]):
            for dh in [-1,0,1]:
                for dw in [-1,0,1]:
                    locH_orig   = int(np.round(landmarks3D_curr[k,1,frame])) + dh
                    locW_orig   = int(np.round(landmarks3D_curr[k,0,frame])) + dw
                    
                    locH_sp     = int(np.round(landmarks3D_curr_rec[k,1,frame])) + dh
                    locW_sp     = int(np.round(landmarks3D_curr_rec[k,0,frame])) + dw
                    
                    locH_sp_tmp = int(np.round(landmarks3D_curr_rec_filtered[k,1,frame])) + dh
                    locW_sp_tmp = int(np.round(landmarks3D_curr_rec_filtered[k,0,frame])) + dw
                    try:
                        imageWithMarkings_orig[locH_orig,locW_orig,channelToMark,frame] = 255
                        imageWithMarkings_sp[locH_sp,locW_sp,channelToMark,frame] = 255
                        imageWithMarkings_sp_tmp[locH_sp_tmp,locW_sp_tmp,channelToMark,frame] = 255
                    except:
                        pass
            
    SideBySide = np.hstack((imageWithMarkings_orig,imageWithMarkings_sp,imageWithMarkings_sp_tmp))
    return SideBySide

In [ ]:
SideBySide = CreateVideosWithMarkingsSideBySide(videoFile['colorImages'], landmarks3D_curr, landmarks3D_curr_rec, landmarks3D_curr_rec_filtered)

#%% show video animations
def WriteColorVideo(video, filename='sample.gif', fps=20):
    writer = imageio.get_writer(filename, fps=fps)
    for frame in range(video.shape[-1]):
        writer.append_data(video[:, :, :, frame])
    writer.close()

#WriteColorVideo(SideBySide,'processingStages.gif',fps=25)

In [ ]:
import imageio
WriteColorVideo(SideBySide,'processingStages.gif',fps=25)

In [ ]:
from PIL import Image
im = Image.open('processingStages.gif')

In [ ]:
MSEC_PER_FRAME    = 40
MSEC_REPEAT_DELAY = 500

# Create an animated GIF file from a sequence of images
def build_gif(inImages, fname=None, show_gif=True, save_gif=True, title=''):
    fig = plt.figure(figsize=(12,5))
    ax = fig.add_subplot(111)
    ax.set_axis_off()
    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, 
                        wspace=None, hspace=None)  # removes white border
    
    imgs = [ (ax.imshow(inImages[:,:,:,frame]), 
              ax.set_title(title), 
              ax.annotate(frame,(5,5))) for frame in range(inImages.shape[3]) ] 

    img_anim = animation.ArtistAnimation(fig, imgs, interval=MSEC_PER_FRAME, 
                                         repeat_delay=MSEC_REPEAT_DELAY, blit=False)
    if save_gif:
        print('Writing:', fname)
        img_anim.save(fname, writer='imagemagick')
    if show_gif:
        plt.show();
    plt.clf() # clearing the figure when done prevents a memory leak 

In [ ]:
titleStr = 'original video | spatially filtered | spatio-temporally filtered'
build_gif(SideBySide, fname='smoothing_stages_side_by_side.gif', show_gif=False, save_gif=True, title=titleStr)

side1=[]
for i in SideBySide:
    side2=[]
    for j in i:
        side2.append(j[:,0])
    side1.append(side2)
    
side1 = np.array(side1)
side1.shape
#(99, 327, 3)

In [ ]:
import numpy as np
side1 = np.empty((SideBySide.shape[0],SideBySide.shape[1],SideBySide.shape[2],SideBySide.shape[3]), dtype=object)
side1 = SideBySide
print(side1.shape)

import numpy as np
import cv2
capture = cv2.VideoCapture(0 )
w=int(capture.get(3))
h=int(capture.get(4))
#fourcc = cv2.VideoWriter_fourcc(*'XVID') 
#video_writer = cv2.VideoWriter("output.avi", fourcc, 25, (w, h))   

while (capture.isOpened()):

    ret, frame = capture.read()
    if ret:
        #print("frame : ", frame)
        print("shape : ", frame.shape)
        #print(type(frame))
      

        #video_writer.write(frame)
        cv2.imshow('Video Stream', frame)

    else:
        break


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
#video_writer.release()
cv2.destroyAllWindows()

In [ ]:
#%% helper function: create videos with keypoints overlaid for each of the 3 processing stages
def CreateVideosWithMarkingsSideBySide(colorImages, landmarks3D_curr, landmarks3D_curr_rec, landmarks3D_curr_rec_filtered):
    imageWithMarkings_orig   = colorImages.copy()
    imageWithMarkings_sp     = colorImages.copy()
    imageWithMarkings_sp_tmp = colorImages.copy()
    
    # paint requested channel
    channelToMark = 1
    for frame in range(colorImages.shape[3]):
        for k in range(landmarks3D_curr.shape[0]):
            for dh in [-1,0,1]:
                for dw in [-1,0,1]:
                    locH_orig   = int(np.round(landmarks3D_curr[k,1,frame])) + dh
                    locW_orig   = int(np.round(landmarks3D_curr[k,0,frame])) + dw
                    
                    locH_sp     = int(np.round(landmarks3D_curr_rec[k,1,frame])) + dh
                    locW_sp     = int(np.round(landmarks3D_curr_rec[k,0,frame])) + dw
                    
                    locH_sp_tmp = int(np.round(landmarks3D_curr_rec_filtered[k,1,frame])) + dh
                    locW_sp_tmp = int(np.round(landmarks3D_curr_rec_filtered[k,0,frame])) + dw
                    try:
                        imageWithMarkings_orig[locH_orig,locW_orig,channelToMark,frame] = 255
                        imageWithMarkings_sp[locH_sp,locW_sp,channelToMark,frame] = 255
                        imageWithMarkings_sp_tmp[locH_sp_tmp,locW_sp_tmp,channelToMark,frame] = 255
                    except:
                        pass
            
    SideBySide = np.hstack((imageWithMarkings_orig,imageWithMarkings_sp,imageWithMarkings_sp_tmp))
    
    SideBySide = imageWithMarkings_sp_tmp
    return SideBySide

In [ ]:
SideBySide = CreateVideosWithMarkingsSideBySide(videoFile['colorImages'], landmarks3D_curr, landmarks3D_curr_rec, landmarks3D_curr_rec_filtered)


In [ ]:
MSEC_PER_FRAME    = 40
MSEC_REPEAT_DELAY = 500

def build_gif(inImages, fname=None, show_gif=True, save_gif=True, title=''):
    fig = plt.figure(figsize=(12,5))
    ax = fig.add_subplot(111)
    ax.set_axis_off()
    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, 
                        wspace=None, hspace=None)  
    
    imgs = [ (ax.imshow(inImages[:,:,:,frame]), 
              ax.set_title(title), 
              ax.annotate(frame,(5,5))) for frame in range(inImages.shape[3]) ] 

    img_anim = animation.ArtistAnimation(fig, imgs, interval=MSEC_PER_FRAME, 
                                         repeat_delay=MSEC_REPEAT_DELAY, blit=False)
    if save_gif:
        img_anim.save(fname, writer='imagemagick')
    if show_gif:
        plt.show();
    plt.clf() 

In [ ]:
titleStr = 'original video | spatially filtered | spatio-temporally filtered'
build_gif(SideBySide, fname='smoothing_stages_side_by_side.gif', show_gif=False, save_gif=True, title=titleStr)

In [ ]:
import cv2
inImages = SideBySide

fig = plt.figure(figsize=(12,5))
ax = fig.add_subplot(111)
ax.set_axis_off()

fourcc = cv2.VideoWriter_fourcc(*'XVID') #########################
video_writer = cv2.VideoWriter("output.avi", fourcc, 10. ,(99,109)) ########################

fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)  
    
imgs = [(ax.imshow(inImages[:,:,:,frame]), ax.set_title("gif"), ax.annotate(frame,(5,5))) for frame in range(inImages.shape[3]) ] 

#[video_writer.write(inImages[:,:,:,frame]) for frame in range(inImages.shape[3])] #####################

[video_writer.write(cv2.resize(inImages[:,:,:,frame],(480,360),interpolation=cv2.INTER_AREA)) for frame in range(inImages.shape[3])]

#[cv2.imshow("eer",cv2.resize(inImages[:,:,:,frame],(480,360),interpolation=cv2.INTER_AREA)) for frame in range(inImages.shape[3])]

cv2.imshow("eer",(inImages[:,:,:,0]))

img_anim = animation.ArtistAnimation(fig, imgs, interval=MSEC_PER_FRAME, repeat_delay=MSEC_REPEAT_DELAY, blit=False)

img_anim.save('smoothing_stages_side_by_side.gif', writer='imagemagick')

#plt.show()
plt.clf() 

video_writer.release() #######################